In [1]:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
from requests import get
url = 'https://www.autotrader.co.uk/car-search?advertClassification=standard&postcode=RH104JJ&make=&price-from=500&price-to=100000&onesearchad=Used&onesearchad=Nearly%20New&onesearchad=New&advertising-location=at_cars&is-quick-search=TRUE&page=1'
response = get(url)
print(response.text[:500])

In [ ]:
from bs4 import BeautifulSoup
html_soup = BeautifulSoup(response.text, 'html.parser')
type(html_soup)

In [ ]:
car_containers = html_soup.find_all('li', class_ = 'search-page__result')
print(type(car_containers))
print(len(car_containers))

In [ ]:
first_car = car_containers[0]
first_car

In [ ]:
first_make = first_car.h2.text.strip().title().split(' ')[0]
first_make

In [ ]:
first_price = first_car.find("div", {"class": "vehicle-price"}).text
first_price

In [ ]:
try:
    first_rating = first_car.find("div", {"class": "js-tooltip"}).find("div", {"class": "pi-indicator js-tooltip-trigger"}).text.strip()
except:
    first_rating = ''
first_rating

In [ ]:
first_specs = first_car.find("ul", {"class": "listing-key-specs"}).find_all("li", recursive=True)

In [ ]:
first_date = first_specs[0].text
first_type = first_specs[1].text
first_miles = first_specs[2].text
first_litre = first_specs[3].text
first_bhp = first_specs[4].text
first_transmission = first_specs[5].text
first_fuel = first_specs[6].text

In [2]:
def scrape(url):
    
    # Lists to store the scraped data in
    makes = []
    prices = []
    ratings = []
    dates = []
    types = []
    miles = []
    litres = []
    bhps = []
    transmissions = []
    fuels = []
    owners = []
    ulez = []

    # Attempt to connect to the url
    try:
        response = get(url)
    except:
        print('oops')
    html_soup = BeautifulSoup(response.text, 'html.parser')

    # Get a list of individual cars and iterate through it
    car_containers = html_soup.find_all('li', class_ = 'search-page__result')
        
    for container in car_containers:
        try:
            container.find("div", {"class": "js-tooltip"}).find("div", {"class": "pi-indicator js-tooltip-trigger"}).text
            rating = container.find("div", {"class": "js-tooltip"}).find("div", {"class": "pi-indicator js-tooltip-trigger"}).text.strip()
        except:
            rating = ''
        ratings.append(rating)
        
        make = container.h2.text.strip().title().split(' ')[0]
        makes.append(make)

        price = container.find("div", {"class": "vehicle-price"}).text[1:]
        prices.append(price)

        specs = container.find("ul", {"class": "listing-key-specs"}).find_all("li", recursive=True)

        for spec in specs:
            
            if spec.text.split(' ')[0].isdigit() and len(spec.text.split(' ')[0]) == 4:
                date = spec.text.split(' ')[0]
                dates.append(date)

            if 'mile' in str(spec):
                mile = spec.text.split(' ')[0]
                miles.append(mile)

            if 'l' in str(spec).lower() and str(spec.text)[:-1].replace('.', '').isnumeric() and not spec.text.split(' ')[0].isdigit():
                litre = spec.text[:-1]
                litres.append(litre)

            if any(x in str(spec).lower() for x in ['automatic', 'manual']):
                transmission = spec.text
                transmissions.append(transmission)

            if any(x in str(spec).lower() for x in ['bhp', 'ps']):
                bhp = spec.text
                bhps.append(bhp)

            if any(x in str(spec).lower() for x in ['petrol', 'diesel']):
                fuel = spec.text
                fuels.append(fuel)
                
            if 'owner' in str(spec).lower():
                owner = spec.text
                owners.append(owner.split(' ')[0])
            
            if 'ulez' in str(spec).lower():
                ulez = spec.text
                ulezs.append(owner.split(' ')[0])
            
        
            typelist = ['hatchback', 'saloon', 'convertible', 'coupe', 'suv', 'mpv', 'estate', 'limousine',
                        'pickup']
            if any(x in str(spec).lower() for x in typelist):
                typ = spec.text
                types.append(typ)
            
        
        attributes = [makes, ratings, dates, types, miles, litres, bhps, transmissions, fuels, owners, ulez]

        # Filling in empty spaces
        for attribute in attributes:
            if len(attribute) < len(prices):
                attribute.append('')
                
    # Creating a dataframe from the lists
    df = ({'makes': makes,
           'Price': prices,
            'Rating': ratings,
            'Year': dates,
            'Type': types,
            'Miles': miles,
            'Litres': litres,
            'BHP': bhps,
            'Transmission': transmissions,
            'Fuel': fuels,
            'Owners': owners,
            'ULEZ': ulezs
    })
    
    return df

In [40]:
from multiprocessing import Pool

# List of urls
path = 'https://www.autotrader.co.uk/car-search?advertClassification=standard&postcode=RH104JJ&make=&price-from=500&price-to=100000&onesearchad=Used&onesearchad=Nearly%20New&onesearchad=New&advertising-location=at_cars&is-quick-search=TRUE&page='
urls = []
for i in range(1,101):
    url = path + str(i)
    urls.append(url)    
    
p = Pool(10)
df = p.map(scrape, urls)
p.terminate()
p.join()

In [57]:
makes = []
prices = []
ratings = []
dates = []
types = []
miles = []
litres = []
bhps = []
transmissions = []
fuels = []
owners = []

for i in range(len(df)):
    for j in range(len(df[i])):
        makes.append(df[i]['makes'][j])
        prices.append(df[i]['Price'][j])
        ratings.append(df[i]['Rating'][j])
        dates.append(df[i]['Year'][j])
        types.append(df[i]['Type'][j])
        miles.append(df[i]['Miles'][j])
        litres.append(df[i]['Litres'][j])
        bhps.append(df[i]['BHP'][j])
        transmissions.append(df[i]['Transmission'][j])
        fuels.append(df[i]['Fuel'][j])
        owners.append(df[i]['Owners'][j])


# Creating a dataframe from the lists
df = ({'makes': makes,
       'Price': prices,
        'Rating': ratings,
        'Year': dates,
        'Type': types,
        'Miles': miles,
        'Litres': litres,
        'BHP': bhps,
        'Transmission': transmissions,
        'Fuel': fuels,
        'Owners': owners
})
df = pd.DataFrame(df)

TypeError: list indices must be integers or slices, not str

In [56]:
df

[[{'makes': ['Seat',
    'Volkswagen',
    'Peugeot',
    'Vauxhall',
    'Citroen',
    'Vauxhall',
    'Honda',
    'Daihatsu',
    'Citroen',
    'Volkswagen',
    'Mercedes-Benz',
    'Renault',
    'Ford'],
   'Price': ['14,050',
    '2,500',
    '695',
    '1,299',
    '595',
    '999',
    '1,250',
    '1,000',
    '2,295',
    '1,995',
    '2,595',
    '13,695',
    '9,950'],
   'Rating': ['Good price',
    'Lower price',
    '',
    '',
    '',
    '',
    '',
    '',
    'Great price',
    'Lower price',
    '',
    'Good price',
    'Good price'],
   'Year': ['2018',
    '2010',
    '2002',
    '2006',
    '2004',
    '2007',
    '2008',
    '2006',
    '2009',
    '2006',
    '2003',
    '2017',
    '2018'],
   'Type': ['Hatchback',
    'Estate',
    'Hatchback',
    'MPV',
    'Hatchback',
    'MPV',
    'Hatchback',
    'Convertible',
    'Hatchback',
    'Hatchback',
    'Convertible',
    'Hatchback',
    'Hatchback'],
   'Miles': ['9,109',
    '123,700',
    '96,000',


In [54]:
df = []
for i in range(1,1001,100):
    for j in range(i,i+100):
        urls = []
        url = path + str(j) 
        urls.append(url)

    p = Pool(20)
    df.append(p.map(scrape, urls))
    p.terminate()
    p.join()
    print(i)

1
101
201
301
401
501
601
701
801
901


In [ ]:
# Import required libraries
from requests import get
from bs4 import BeautifulSoup
import pandas as pd

# List of urls
path = 'https://www.autotrader.co.uk/car-search?advertClassification=standard&postcode=RH104JJ&make=&price-from=500&price-to=100000&onesearchad=Used&onesearchad=Nearly%20New&onesearchad=New&advertising-location=at_cars&is-quick-search=TRUE&page='
urls = []
for i in range(1,1000):
    url = path + str(i)
    urls.append(url)

# Lists to store the scraped data in
makes = []
prices = []
ratings = []
dates = []
types = []
miles = []
litres = []
bhps = []
transmissions = []
fuels = []
owners = []

attributes = [makes, ratings, dates, types, miles, litres, bhps, transmissions, fuels, owners]
    
# Iterate through urls
sum = 0
for url in urls:
    sum += 1
    if sum%10 == 0:
        print(sum)
    
    # Attempt to connect to the url
    try:
        response = get(url)
    except:
        print('oops')
    html_soup = BeautifulSoup(response.text, 'html.parser')

    # Get a list of individual cars and iterate through it
    car_containers = html_soup.find_all('li', class_ = 'search-page__result')
        
    for container in car_containers:
        try:
            container.find("div", {"class": "js-tooltip"}).find("div", {"class": "pi-indicator js-tooltip-trigger"}).text
            rating = container.find("div", {"class": "js-tooltip"}).find("div", {"class": "pi-indicator js-tooltip-trigger"}).text.strip()
        except:
            rating = ''
        ratings.append(rating)
        
        make = container.h2.text.strip().title().split(' ')[0]
        makes.append(make)

        price = container.find("div", {"class": "vehicle-price"}).text[1:]
        prices.append(price)

        specs = container.find("ul", {"class": "listing-key-specs"}).find_all("li", recursive=True)

        for spec in specs:
            
            if spec.text.split(' ')[0].isdigit() and len(spec.text.split(' ')[0]) == 4:
                date = spec.text.split(' ')[0]
                dates.append(date)

            if 'mile' in str(spec):
                mile = spec.text.split(' ')[0]
                miles.append(mile)

            if 'l' in str(spec).lower() and str(spec.text)[:-1].replace('.', '').isnumeric() and not spec.text.split(' ')[0].isdigit():
                litre = spec.text[:-1]
                litres.append(litre)

            if any(x in str(spec).lower() for x in ['automatic', 'manual']):
                transmission = spec.text
                transmissions.append(transmission)

            if any(x in str(spec).lower() for x in ['bhp', 'ps']):
                bhp = spec.text
                bhps.append(bhp)

            if any(x in str(spec).lower() for x in ['petrol', 'diesel']):
                fuel = spec.text
                fuels.append(fuel)
                
            if 'owner' in str(spec):
                owner = spec.text
                owners.append(owner.split(' ')[0])
            
        
            typelist = ['hatchback', 'saloon', 'convertible', 'coupe', 'suv', 'mpv', 'estate', 'limousine',
                        'pickup']
            if any(x in str(spec).lower() for x in typelist):
                typ = spec.text
                types.append(typ)
            
        
        # Filling in empty spaces
        for attribute in attributes:
            if len(attribute) < len(prices):
                attribute.append('')
                    

# Creating a dataframe from the lists
df = ({'makes': makes,
       'Price': prices,
        'Rating': ratings,
        'Year': dates,
        'Type': types,
        'Miles': miles,
        'Litres': litres,
        'BHP': bhps,
        'Transmission': transmissions,
        'Fuel': fuels,
        'Owners': owners
})
df = pd.DataFrame(df)

In [ ]:
#df.to_csv('car_data')

In [ ]:
df

In [ ]:
makes = []
prices = []
ratings = []
dates = []
types = []
miles = []
litres = []
bhps = []
transmissions = []
fuels = []

first_specs = first_car.find("ul", {"class": "listing-key-specs"}).find_all("li", recursive=True)

for spec in first_specs:
    if spec.text.split(' ')[0].isdigit() and len(spec.text.split(' ')[0]) == 4 and 'miles' not in spec:
        date = spec.text.split(' ')[0]
        dates.append(date)
    
    if 'miles' in str(spec):
        mile = spec.text
        miles.append(mile)
        
    if 'L' in str(spec):
        litre = spec.text
        litres.append(litre)
    
    if any(x in str(spec).lower() for x in ['automatic', 'manual']):
        transmission = spec.text
        transmissions.append(transmission)
    
    if any(x in str(spec).lower() for x in ['bhp', 'ps']):
        bhp = spec.text
        bhps.append(bhp)
        
    if any(x in str(spec).lower() for x in ['petrol', 'diesel']):
        fuel = spec.text
        fuels.append(fuel)

In [ ]:
transmissions

In [ ]:
typ = specs[1].text
        types.append(specs[1].text)